In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install flask flask-ngrok torch transformers diffusers peft ngrok flask_cors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 87.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
import io
import subprocess
from flask import Flask, request, send_file, abort
from flask_cors import CORS
import torch
from PIL import Image
from diffusers import (
    StableDiffusionImg2ImgPipeline,
    DPMSolverMultistepScheduler,
    AutoencoderKL,
    UNet2DConditionModel
)
from transformers import CLIPTextModel, CLIPTokenizer
from huggingface_hub import login

# 1. Install dependencies
def install_dependencies():
    subprocess.run(
        ['pip', 'install', 'pyngrok', 'flask', 'torch', 'diffusers', 'transformers',
         'accelerate', 'peft', 'pillow'],
        check=True
    )

install_dependencies()

# 2. Flask app setup
app = Flask(__name__)
CORS(app)

# 3. Ngrok setup (optional)
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("")
public_url = ngrok.connect(5051).public_url
print(f" * Ngrok tunnel: {public_url}")

# 4. Initialize SD2.1 pipeline with explicit components
pipe = None
def initialize_pipeline():
    global pipe
    login(token="")

    model_id = "stabilityai/stable-diffusion-2-1"

    # Load components explicitly :cite[1]:cite[3]
    text_encoder = CLIPTextModel.from_pretrained(
        model_id, subfolder="text_encoder",
        torch_dtype=torch.float16
    )
    tokenizer = CLIPTokenizer.from_pretrained(
        model_id, subfolder="tokenizer"
    )
    vae = AutoencoderKL.from_pretrained(
        model_id, subfolder="vae",
        torch_dtype=torch.float16
    )
    unet = UNet2DConditionModel.from_pretrained(
        model_id, subfolder="unet",
        torch_dtype=torch.float16
    )

    # Initialize pipeline with custom components :cite[5]
    pipe = StableDiffusionImg2ImgPipeline(
        vae=vae,
        text_encoder=text_encoder,
        tokenizer=tokenizer,
        unet=unet,
        scheduler=DPMSolverMultistepScheduler.from_pretrained(model_id, subfolder="scheduler"),
        safety_checker=None,
        feature_extractor=None,
        requires_safety_checker=False
    ).to("cuda")

    pipe.enable_attention_slicing()

initialize_pipeline()

# 5. Modified /generate endpoint
@app.route('/generate', methods=['POST'])
def generate():
    if 'prompt' not in request.form or 'image' not in request.files:
        abort(400, description="Missing 'prompt' or 'image'")

    prompt = request.form['prompt']
    init_file = request.files['image']

    try:
        init_image = Image.open(init_file).convert("RGB").resize((768, 768))  # SD2.1-768 requires 768px :cite[2]
    except Exception as e:
        abort(400, description=f"Image error: {e}")

    output = pipe(
        prompt=prompt,
        image=init_image,
        strength=0.7,
        guidance_scale=7.5,
        num_inference_steps=50
    )

    img_io = io.BytesIO()
    output.images[0].save(img_io, format='PNG')
    img_io.seek(0)
    return send_file(img_io, mimetype='image/png')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5051)

 * Ngrok tunnel: https://c6fb-104-197-100-49.ngrok-free.app


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.36G [00:00<?, ?B/s]

...

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5051
 * Running on http://172.28.0.12:5051
INFO:werkzeug:Press CTRL+C to quit
Token indices sequence length is longer than the specified maximum sequence length for this model (91 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['aesthetic poster design , instagram ad style , high - resolution , commercial look']


  0%|          | 0/35 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [16/May/2025 00:35:48] "POST /generate HTTP/1.1" 200 -
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['aesthetic poster design , instagram ad style , high - resolution , commercial look']


  0%|          | 0/35 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [16/May/2025 00:36:03] "POST /generate HTTP/1.1" 200 -
